In [8]:
import pandas as pd
import numpy as np 
from util import get_split_cols, get_split_frame


In [9]:
train = pd.read_csv("/data/project2/train.csv")

test = pd.read_csv("/data/project2/test.csv")

In [14]:
num_train_cols, cat_train_cols = get_split_cols(train)
num_test_cols, cat_test_cols = get_split_cols(test)

In [15]:
assert(len(num_train_cols)==len(num_test_cols))
assert(len(cat_train_cols)==len(cat_test_cols))

In [17]:
num_train_df, cat_train_df = get_split_frame(train)
num_test_df, cat_test_df = get_split_frame(test)

In [18]:
y = num_train_df['Total Household Income']

In [124]:
class Neuron(object):
    def __init__(self, weights):
        self.output = None
        self.delta = None
        self.weights = weights

class NeuralNetwork(object):
    def __init__(self, num_inputs, hidden_nodes, num_outputs, eta=.005, activation='sigmoid'):
        self.outputs = None
        self.hidden_index = 0
        self.output_index = 1
        self.sum_error = 0
        self.num_hidden = hidden_nodes
        self.num_outputs = num_outputs
        self.hidden_layer_section = self.create_randomized_layer_section(num_inputs, hidden_nodes)
        self.output_layer_section = self.create_randomized_layer_section(hidden_nodes, num_outputs - 1)
        self.network = [self.hidden_layer_section, self.output_layer_section]
        self.activation_type = activation
        self.eta = eta
        
    def create_randomized_layer_section(self, prev_nodes, num_nodes):
        return [Neuron(np.random.rand(prev_nodes+1)) for _ in range(num_nodes + 1)]
    
    def activation(self, neuron, prev_x):
        z = self.calc_z(neuron, prev_x)
        if self.activation == 'sigmoid':
            return 1 / (1 - exp(-z))
        else if self.activation == 'tanh':
            return np.tanh(z)
        else if self.activation == 'relu':
            if z > 0: 
                return z
            else:
                return 0 
        else:
            return -1 
        
    def d_activation(self, value):
        if self.activation == 'sigmoid':
            return value * (1 - value)
        else if self.activation == 'tanh':
            return 1 - value ** 2 
        else if self.activation == 'relu':
            if value > 0: 
                return 1
            else:
                return 0 
        else:
            return -1 
        
    def calc_z(self, neuron, prev_x):
        return sum([x * w for x,w in zip(prev_x, neuron.weights)])
    
        
    def calc_output(self, neuron, prev_x, i):
        if i == self.output_index:
            return self.calc_z(neuron, prev_x)
        return self.activation(neuron, prev_x)
    
    def output_delta(self, neuron, expected):
        return expected - neuron.output
        
    def hidden_delta(self, neuron, n_idx, next_neuron):
        return next_neuron.weights[n_idx] * next_neuron.delta * self.d_activation(neuron.output) 
        
    def forward(self, x):
        for i, layer in enumerate(self.network):
            for n in layer:
                if i == 0:
                    prev_layer = x
                else: 
                    prev_layer = [x.output for x in self.network[i-1]]
                n.output = self.calc_output(n, prev_layer, i)
                
        # print(self.network[1][0].output)
        
        
    def back_prop(self, y):
        for layer_idx, layer in reversed(list(enumerate(self.network))):
            if layer_idx == self.output_index:
                for n in layer:
                    #output layer deltas
                    n.delta = self.output_delta(n, y) #NEED n index 
            else: 
                next_layer = self.network[layer_idx+1]
                for n_i, n in enumerate(layer):
                    #hidden deltas 
                    n.delta = 0
                    for next_neuron in next_layer: 
                        n.delta += self.hidden_delta(n, n_i, next_neuron) 
    
    def update(self):
        for layer_index, layer in enumerate(self.network):
            for neuron in layer:
                neuron.weights -= self.eta * neuron.delta      

    def train(self, x, y, epoch):
        for e in range(epoch):
            self.forward(x)
            self.back_prop(y)
            self.update()
            print(self.network[self.output_index][0].output)

SyntaxError: invalid syntax (<ipython-input-124-acd3aecb6eb4>, line 28)

In [123]:
nn = NeuralNetwork(2, 2, 1, .05)
nn.forward([1, 1])
nn.back_prop( 1)
nn.train([5,1], 1, 100)

-0.880192890965
-0.59816395732
-0.358439363722
-0.154673459164
0.0185275597108
0.165748425754
0.290886161891
0.397253237607
0.487665251966
0.564515464171
0.629838144546
0.685362422864
0.732558059434
0.772674350519
0.806773197941
0.83575721825
0.860393635513
0.881334590186
0.899134401658
0.914264241409
0.927124605198
0.938055914418
0.947347527255
0.955245398167
0.961958588442
0.967664800176
0.972515080149
0.976637818127
0.980142145408
0.983120823597
0.985652700057
0.987804795049
0.989634075791
0.991188964423
0.992510619759
0.993634026795
0.994588922776
0.99540058436
0.996090496706
0.9966769222
0.99717538387
0.997599076289
0.997959214846
0.998265332619
0.998525532726
0.998746702817
0.998934697395
0.999094492785
0.999230318868
0.999345771038
0.999443905382
0.999527319575
0.999598221638
0.999658488393
0.999709715134
0.999753257864
0.999790269184
0.999821728807
0.999848469486
0.999871199063
0.999890519203
0.999906941323
0.999920900124
0.999932765106
0.99994285034
0.999951422789
0.9999587093